# scikits.statsmodels notebook for AY250<br>  Berian James, 24-Sep-2012

**I. Regression (OLS)**

Import command. Note use of API request.

In [ ]:
import numpy as np
import statsmodels.api as sm

Build a simple data set $y = a_1x^2 + a_2x + a_3 + \epsilon$ with $\epsilon\sim N(0,1)$

In [ ]:
npts = 100
x = np.linspace(0,10, npts)

In [ ]:
X = sm.add_constant(np.column_stack((x**2,x)),prepend=False)

In [ ]:
beta = np.array([0.1, 1, 10])
y = np.dot(X, beta) + np.random.normal(size=npts)

In [ ]:
plot(x,y)

Now, fit the model for the coefficients

In [ ]:
results = sm.OLS(y, X).fit()

Examine formatted summary of results

In [ ]:
print results.summary()

In [ ]:
plot(x,y,x,results.fittedvalues)

In [ ]:
print results.params
print results.normalized_cov_params

Trending and detrending

In [ ]:
yp = sm.tsa.detrend(y,order=1)

In [ ]:
plot(x,yp)

**II. Regression (WLS)**

In [ ]:
w = 0.1*(101-x**2)
y = np.dot(X, beta) + np.random.normal(scale=w,size=npts)
plot(x,y)

In [ ]:
ols_results = sm.OLS(y,X).fit()

In [ ]:
wls_model = sm.WLS(y,X, weights=1./w**2)
wls_results = wls_model.fit()
wls_results.params

In [ ]:
plot(x,y,x,ols_results.fittedvalues,x,wls_results.fittedvalues)

In [ ]:
subplot(211)
plot(x,wls_results.resid)
subplot(212)
plot(x,ols_results.resid)

**III. Rudimentary time-series analysis**

In [ ]:
import scipy.io as sio
EEG = sio.loadmat('eeglab_data.set',struct_as_record=True)

Build x-axis data

In [ ]:
npts = (EEG['EEG'])['pnts']
xmax = (EEG['EEG'])['xmax']
t = linspace(0,xmax,npts)

Select channel

In [ ]:
channels = (EEG['EEG'])['data'][0,0]
y = channels[15,:]
plot(y)

Take autocorrelation

In [ ]:
acf = sm.tsa.acf(y)
bar(range(41),acf,width=0.2)

In [ ]:
acovf = sm.tsa.acovf(y)
f = plot(acovf)

In [ ]:
_ = acorr(y,maxlags=50)

In [ ]:
Py = sm.tsa.periodogram(y)
plot(Py)

**IV. Vector auto-regressive processes**

Assume one has $T$ observations of $k$ variables. A vector auto-regressive process attempts to describe the relationship between the variables from their previous values, so that:

$$\mathbf{y}(t) = \sum_{i=1}^{p} \mathbf{A}_i \mathbf{y}(t-i) + \epsilon(\mathbf{0},\Sigma)$$

where $\epsilon$ is normally distributed and each $A_i$ is a $k\times k$ matrix of coefficients for that time location. This can be expressed more concisely as the matrix equation:

$$Y = BZ + U$$

where $Y$ is $k\times T$, $B$ is $pk\times k$, and $Z$ and $U$ are $k\times T$. The process solves for the coefficient matrix $B$ and its covariance:

$$\hat{B} = YZ'(ZZ')^{-1}$$
$$ \textbf{Cov}(\hat{B}) = (ZZ')^{-1} \otimes \hat\Sigma, \textrm{ where } \hat\Sigma = \frac{1}{T-kp-1}(Y-\hat{B}Z)(Y-\hat{B}Z)'$$

Let's load some macroeconomic data

In [ ]:
mdata = sm.datasets.macrodata.load().data
mdata = mdata[['realgdp','realcons','realinv']]
names = mdata.dtype.names

In [ ]:
data = mdata.view((float,3))
sm.tsa.var.plotting.plot_mts(data)

*Technical point!* VAR processes work best for stationary data, so we should use a numerical differencing method to cast our input data into this form.

In [ ]:
data = np.diff(np.log(data), axis=0)
sm.tsa.var.plotting.plot_mts(data)

Construct the model

In [ ]:
model = sm.tsa.VAR(data)

In [ ]:
lags = 10
results = model.fit(lags)

Forecasting

In [ ]:
#results = model.fit(maxlags=lags, ic='aic')
nfore = 4
lower, forecast, upper = results.forecast_interval(data, nfore)

In [ ]:
sm.tsa.var.plotting.plot_var_forc(data,forecast,upper,lower)

See:
http://en.wikipedia.org/wiki/Christopher_A._Sims<br>
http://en.wikipedia.org/wiki/Nobel_prize_economics#Alternative_names